In [1]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#opening connection & cursor
conn = psycopg2.connect(
                            host="lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com",
                            database="mid_term_project",
                            user="lhl_student",
                            password="lhl_student")
cursor = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [3]:
df = pd.DataFrame()

for i in range(1, 13):
    if i == 1:
        test2 = f"SELECT * FROM flights WHERE arr_delay IS NOT NULL AND weather_delay IS NOT NULL AND arr_delay < 200 AND EXTRACT(month from CAST(fl_date as date)) = {i} AND EXTRACT(year from CAST(fl_date as date)) = 2019 LIMIT 10000;"
        df = create_pandas_table(test2)
    else:
        test2 = f"SELECT * FROM flights WHERE arr_delay IS NOT NULL AND weather_delay IS NOT NULL AND arr_delay < 200 AND EXTRACT(month from CAST(fl_date as date)) = {i} AND EXTRACT(year from CAST(fl_date as date)) = 2019 LIMIT 10000;"
        test_pd_table = create_pandas_table(test2)
        df = pd.concat([df, test_pd_table]) 

In [4]:
passengers = create_pandas_table("SELECT * FROM passengers TABLESAMPLE SYSTEM(10) WHERE year >= 2019 ")

In [5]:
passengers.shape

(47057, 38)

In [6]:
passengers.head()

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,0.0,1.0,4888.0,12.0,5.0,0.0,0.0,3263.0,0.0,0.0,...,GB,United Kingdom,7,725,1,2019,6,7,L,IF
1,0.0,1.0,3201.0,14.0,1.0,0.0,0.0,4784.0,0.0,0.0,...,DK,Denmark,7,775,1,2019,6,10,L,IF
2,0.0,1.0,3201.0,14.0,1.0,0.0,0.0,3945.0,0.0,0.0,...,US,United States,7,775,1,2019,6,8,L,IF
3,0.0,1.0,4888.0,12.0,1.0,0.0,0.0,3747.0,0.0,0.0,...,US,United States,7,725,1,2019,6,8,L,IF
4,0.0,1.0,3201.0,12.0,1.0,0.0,0.0,3927.0,0.0,0.0,...,US,United States,7,775,1,2019,6,8,L,IF


In [7]:
fuel = create_pandas_table("SELECT * FROM fuel_comsumption WHERE year >= 2019 ")

In [8]:
fuel.shape

(458, 25)

In [9]:
fuel_mean=fuel[['unique_carrier','total_gallons']].groupby(['unique_carrier'],as_index=False).mean()

In [10]:
fuel_mean.rename(columns={'unique_carrier':'mkt_unique_carrier'},inplace=True)


In [11]:
passengers['capacity'] = passengers['seats'] - passengers['passengers']

In [12]:
passengers_mean=passengers[['unique_carrier', 'origin','dest', 'passengers', 'capacity']].groupby(['unique_carrier', 'origin','dest'],as_index=False).mean()


In [13]:
passengers_mean.rename(columns={'unique_carrier':'mkt_unique_carrier'},inplace=True)


In [ ]:
df_merged=pd.merge(left=df,right=passengers_mean,how='left',on=['mkt_unique_carrier'],) # Merge flights with passengers

In [ ]:
df_merged=pd.merge(left=df_merged,right=fuel_mean,how='left',on=['mkt_unique_carrier'],) # Merge flights with passengers

In [ ]:
df_merged.head()

In [ ]:
df_merged.isna().sum()

In [ ]:
# remember to close connection to allow for others to use the bandwidth
cursor.close()
conn.close()

In [ ]:
df_merged.passengers.unique()

In [ ]:
df_merged.to_csv('flights_merged.csv')